In [1]:
import sympy
sympy.init_printing()

In [2]:
from itertools import chain, count
from functools import reduce

from src import row

In [3]:
def mathsum(f, args):
    return sum(map(f, range(*args)))


def grid_length(width, margin, radius_scale, total_lines):
    radius = radius_scale(0, total_lines)

    return sum(
        row.length(width, margin, radius(line_number))
        for line_number in range(total_lines)
    )

In [4]:
width = sympy.Symbol('width', positive=True, real=True)
margin = sympy.Symbol('margin', positive=True, real=True)

maxRadius = sympy.Symbol('maxRadius', positive=True, real=True)
minRadius = sympy.Symbol('minRadius', positive=True, real=True)

i = sympy.Symbol('i', negative=False, integer=True)
t = sympy.Symbol('t', positive=False, integer=True)

radius = sympy.Function('radius', positive=True, real=True)

rel = (
    sympy.Symbol('nodeCount', positive=True, integer=True) <=
    sympy.Sum(
        row.length(width, margin, radius(maxRadius, minRadius, 0, t, i)),
        (i, 0, t)
    )
)

In [5]:
rel

              t                                                     
             ____                                                   
             ╲                                                      
              ╲   ⎢                -margin + width                 ⎥
               ╲  ⎢────────────────────────────────────────────────⎥
nodeCount ≤    ╱  ⎣margin + 2⋅radius(maxRadius, minRadius, 0, t, i)⎦
              ╱                                                     
             ╱                                                      
             ‾‾‾‾                                                   
            i = 0                                                   

In [6]:
sympy.ImageSet(sympy.Lambda(t, rel), sympy.Naturals)

⎧              t                                                             ⎫
⎪             ____                                                           ⎪
⎪             ╲                                                              ⎪
⎪              ╲   ⎢                -margin + width                 ⎥        ⎪
⎪               ╲  ⎢────────────────────────────────────────────────⎥        ⎪
⎨nodeCount ≤    ╱  ⎣margin + 2⋅radius(maxRadius, minRadius, 0, t, i)⎦ | t ∊ ℕ⎬
⎪              ╱                                                             ⎪
⎪             ╱                                                              ⎪
⎪             ‾‾‾‾                                                           ⎪
⎪            i = 0                                                           ⎪
⎩                                                                            ⎭

In [7]:
linscale = sympy.S('y_i + ((y_f - y_i) / (x_f - x_i)) * x - y')

r = linscale.subs(dict(
    y_i=maxRadius, y_f=minRadius,
    x_i=0, x_f=t - 1,
    
    y=0, x=i
))

r

i⋅(-maxRadius + minRadius)            
────────────────────────── + maxRadius
          t - 1                       

In [8]:
sympy.solveset(
    sympy.Eq(
        sympy.Symbol('nodeCount', positive=True, integer=True),
        sympy.Sum(
            row.length(width, margin, radius(maxRadius, minRadius, 0, t, i)),
            (i, 0, t)
        )
    ).subs(radius(maxRadius, minRadius, 0, t, i), r).doit(),
    t,
    sympy.Naturals
)

⎧                           t                                                 
⎪                         _____                                               
⎪                         ╲                                                   
⎪                          ╲    ⎢                  -margin + width            
⎪                           ╲   ⎢─────────────────────────────────────────────
⎪                            ╲  ⎢2⋅i⋅(-maxRadius + minRadius)                 
⎨t | t ∊ ℕ ∧ -nodeCount +    ╱  ⎢──────────────────────────── + margin + 2⋅max
⎪                           ╱   ⎣           t - 1                             
⎪                          ╱                                                  
⎪                         ╱                                                   
⎪                         ‾‾‾‾‾                                               
⎪                         i = 0                                               
⎩                                                   